In [2]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical

In [17]:
env_name = 'CartPole-v0'
env = gym.make(env_name)


# Hard coded policy for the cartpole problem
# Will eventually want to build up infrastructure to develop a policy depending on:
# env.action_space
# env.observation_space

policy = nn.Sequential(
    nn.Linear(4, 12),
    nn.ReLU(),
    nn.Linear(12,12),
    nn.ReLU(),
    nn.Linear(12,2),
    nn.Softmax(dim=-1)
    )

optimizer = optim.Adam(policy.parameters(), lr = .1)

# I guess we'll start with a categorical policy
# TODO investigate the cost of action.detach.numpy() and torch.Tensor(state)
def select_action(policy, state):
    m = Categorical(policy(torch.Tensor(state)))
    action = m.sample()
    logprob = m.log_prob(action)
    
    return action.detach().numpy(), logprob
    

In [3]:
policy(torch.randn(1,4))

tensor([[0.4328, 0.5672]], grad_fn=<SoftmaxBackward>)

In [45]:
#def vanilla_policy_grad(env, policy, optimizer):
    
action_list = []
state_list = []
logprob_list = []
reward_list = []

avg_reward_hist = []



num_epochs = 1000
batch_size = 20 # how many steps we want to use before we update our gradients
num_steps = 100 # number of steps in an episode (unless we terminate early)

loss = torch.zeros(1,requires_grad=True)



for epoch in range(num_epochs):

    loss_hist = []
    episode_length_hist = []
    total_steps = 0

    while True:

        state = env.reset()
        
        for t in range(num_steps):

            action, logprob = select_action(policy, state)
            state, reward, done, _ = env.step(action.item())

            loss_hist.append(logprob*reward)
            total_steps += 1

            if done:
                break
                
        episode_length_hist.append(t)

        if total_steps > batch_size:
            #print("here")
            avg_reward_hist.append(sum(episode_length_hist)/len(episode_length_hist))
            #other_list.append(1)
            loss = torch.sum(torch.stack(loss_hist))
            loss.backward()
            optimizer.step()
            break

In [46]:
while True:
    state = env.reset()
    cum_rewards = 0


    for t in range(num_steps):
        action, _ = select_action(policy,state)
        state, reward, done, _ = env.step(action.item())
        env.render()
        
        cum_rewards += reward
        if done:
            
            print('summed reward for espide: ', cum_rewards)
            print('time terminated:' , t)
            break
        
    

summed reward for espide:  9.0
time terminated: 8
summed reward for espide:  8.0
time terminated: 7
summed reward for espide:  10.0
time terminated: 9
summed reward for espide:  10.0
time terminated: 9
summed reward for espide:  9.0
time terminated: 8
summed reward for espide:  9.0
time terminated: 8
summed reward for espide:  10.0
time terminated: 9
summed reward for espide:  8.0
time terminated: 7
summed reward for espide:  9.0
time terminated: 8
summed reward for espide:  10.0
time terminated: 9
summed reward for espide:  10.0
time terminated: 9
summed reward for espide:  10.0
time terminated: 9
summed reward for espide:  10.0
time terminated: 9
summed reward for espide:  11.0
time terminated: 10
summed reward for espide:  11.0
time terminated: 10
summed reward for espide:  9.0
time terminated: 8
summed reward for espide:  10.0
time terminated: 9
summed reward for espide:  9.0
time terminated: 8
summed reward for espide:  9.0
time terminated: 8
summed reward for espide:  9.0
time te

KeyboardInterrupt: 